In [24]:
import json
import copy
import time

In [3]:
with open('wholegraphtreerepo.json', 'r') as f:
    wholegraphtreerepo = json.load(f)

In [4]:
topicdict = {}
topicnumdesc = {}
def dfs(tree):
    if tree['id'] in topicdict:
        return
    if tree['id'].startswith('tp'):
        topicdict[tree['id']] = tree['label']
    if 'children' in tree:
        for child in tree['children']:
            dfs(child)
dfs(wholegraphtreerepo)
with open('../Topic-Wiki-Valid/topic_desc.json', 'r') as f:
    topic_desc = json.load(f)
for topicnum, topic in topicdict.items():
    if topic in topic_desc:
        topicnumdesc[topicnum] = topic_desc[topic]
with open('topicnumdesc.json', 'w') as f:
    json.dump(topicnumdesc, f)

In [6]:
with open('../Wiki-Github-Link/Pageleaf-encode.json', 'r') as f:
    pageleaf_encode = json.load(f)
with open('Pagedescs.json', 'r', encoding='utf-8') as f:
    pagedescs = json.load(f)
pageinfo = {}
for page, desc in pagedescs.items():
    pageinfo[page] = {
        'desc': desc,
        'name': pageleaf_encode[page]
    }
with open('pageinfo.json', 'w') as f:
    json.dump(pageinfo, f, indent=1)

In [7]:
topicinfo = {}
for topicnum, topic in topicdict.items():
    if topic in topic_desc:
        topicinfo[topicnum] = {'name': topic, 'desc': topic_desc[topic]}
with open('topicinfo.json', 'w') as f:
    json.dump(topicinfo, f, indent=1)

In [8]:
with open('topicdict.json', 'w') as f:
    json.dump(topicdict, f, indent=1)

In [2]:
with open('cateinfoonlygithub.json', 'r') as f:
    cateinfoonlygithub = json.load(f)
with open('pageinfo.json', 'r') as f:
    pageinfo = json.load(f)
knowpdict = {}
for cate, info in cateinfoonlygithub.items():
    if not info['name'] in knowpdict:
        knowpdict[info['name']] = cate
    for page in info['pages']:
        if not pageinfo[str(page)]['name'] in knowpdict:
            knowpdict[pageinfo[str(page)]['name']] = 'pg' + str(page)
with open('knowpdict.json', 'w') as f:
    json.dump(knowpdict, f, indent=1)

In [28]:
topicdict = {}
topicset = set()
def dfs(tree):
    if tree['id'] in topicset:
        return
    topicset.add(tree['id'])
    if not tree['label'] in knowpdict:
        knowpdict[tree['label']] = tree['id']
    if tree['id'].startswith('tp'):
        topicdict[tree['label']] = tree['id']
    if 'children' in tree:
        for child in tree['children']:
            dfs(child)
dfs(wholegraphtreerepo)
with open('knowpdict.json', 'w') as f:
    json.dump(knowpdict, f, indent=1)

In [5]:
knowpdict = {k:v for k,v in knowpdict.items() if not v.startswith('rp')}
with open('knowpdict.json', 'w') as f:
    json.dump(knowpdict, f, indent=1)

In [25]:
cut = set()
totcnt = 0
copycost = 0
def dfsknowgraph(tree, id, repos, depth = -1):
    if tree['id'] in cut or tree['id'].startswith('rp'):
        return None
    global totcnt
    global copycost
    totcnt += 1
    start = time.time()
    newtree = copy.deepcopy(tree)
    copycost += time.time() - start
    cut.add(tree['id'])
    if depth == 0:
        for child in tree['children']:
            if child['id'].startswith('rp'):
                if child['id'] in repos:
                    newtree['children'].remove(child)
            else:
                newtree['children'].remove(child)
        newtree['children'] = newtree['children'][:5]
        return newtree
    
    if depth == 1:
        cnt = 0
        newtree['children'] = []
        newrepos = copy.deepcopy(repos)
        for child in tree['children']:
            if child['id'].startswith('rp') and not child['id'] in repos:
                newtree['children'].append(child)
                cnt += 1
                newrepos.append(child['id'])
                if cnt == 5:
                    break
        for child in tree['children']:
            if not child['id'].startswith('rp'):
                newtree['children'].append(dfsknowgraph(child, child['id'], newrepos, 0))
        return newtree
    
    if depth == 2:
        cnt = 0
        newtree['children'] = []
        newrepos = []
        for child in tree['children']:
            if child['id'].startswith('rp'):
                newtree['children'].append(child)
                cnt += 1
                newrepos.append(child['id'])
                if cnt == 5:
                    break
        for child in tree['children']:
            if not child['id'].startswith('rp'):
                newtree['children'].append(dfsknowgraph(child, child['id'], newrepos, 0))
        return newtree

    if tree['id'] == id:
        cnt = 0
        newtree['children'] = []
        newrepos = []
        for child in tree['children']:
            if child['id'].startswith('rp'):
                newtree['children'].append(child)
                cnt += 1
                newrepos.append(child['id'])
                if cnt == 5:
                    break
        for child in tree['children']:
            if not child['id'].startswith('rp'):
                newtree['children'].append(dfsknowgraph(child, child['id'], newrepos, 1))
        return newtree
    if not 'children' in tree:
        return None
    for child in tree['children']:
        if child['id'] in cut:
            continue
        if (child['id'].startswith('tp') or child['id'].startswith('ts')) and (id.startswith('pg') or id[0].isdigit()):
            continue
        if child['id'].startswith('pg') and id[0].isdigit():
            continue
        if child['id'] == id:
            newtree['children'] = [dfsknowgraph(child, child['id'], [], 2)]
            return newtree
        else:
            dfsre = dfsknowgraph(child, id, [], -1)
            if dfsre != None:
                print(tree['id'])
                return dfsre
    return None
graph = dfsknowgraph(wholegraphtreerepo, 'pg23363', [])
print(graph)
print('totcnt', totcnt)
print('copycost', copycost)

107
7
0
{'id': '932', 'label': 'Java development tools', 'group': 0, 'children': [{'id': 'pg23363', 'label': 'Javadoc', 'group': 1, 'children': [{'id': 'tp7373', 'label': 'javadoc', 'children': [{'id': 'rp71577', 'label': 'pedroslopez/whatsapp-web.js', 'group': 4}, {'id': 'rp89099', 'label': 'eternnoir/pyTelegramBotAPI', 'group': 4}, {'id': 'rp117159', 'label': 'telegraf/telegraf', 'group': 4}, {'id': 'rp89931', 'label': 'javaparser/javaparser', 'group': 4}, {'id': 'rp79785', 'label': 'DV8FromTheWorld/JDA', 'group': 4}], 'group': 2}]}]}
totcnt 7323
copycost 18.57514262199402


In [27]:
wholedict = {}
def dfs(tree, parent):
    if tree['id'].startswith('rp'):
        return
    if tree['id'] in wholedict:
        wholedict[tree['id']]['parent'].append(parent)
        return
    curitem = {'label': tree['label'], 'group': tree['group'], 'parent': [parent]}
    curchildren = [child['id'] for child in tree['children']]
    curitem['children'] = curchildren
    wholedict[tree['id']] = curitem
    for child in tree['children']:
        dfs(child, tree['id'])
dfs(wholegraphtreerepo, None)
with open('wholedict.json', 'w') as f:
    json.dump(wholedict, f, indent=1)